In [117]:
from typing import Sequence, Optional, List

import clingo
import clingo.ast

In [118]:
program = """

domA(0).

domA(A) :- domA(A).

"""

In [119]:
unground_reified = """

% Literal/Disjunctive/Aggregate share ID-Space
% Term/Function/Variable share ID-Space

% domA(0).
rule(1).                  % rule - identifier

head(1).                  % head - identifier
literal(1).               % literal - identifier
literal_sign(1, 0).       % sign = how many nots
literal_atom(1, 1).       % link to atom 1
atom(1).                  % atom - identifier
atom(1, 1).               % link to symbol 1 (function/variable/term)
function(1).              % symbol(function) - identifier
function_name(1, 1).      % name of function
name(1).                  % name - identifier
name_repr(1, "domA").     % repr of name
function_args(1, 1).      % link to argument tuple
args(1).                  % argument tuple - identifier
args_pos_symb(1, 1, 2).   % argument tuple 1 has link to symbol 2 at position 1
term(2).                  % symbol(term) - identifier
term_repr(2, 0).          % representation of term
head(1,1).                % head 1 has link to literal 1

rule_head_body(1, 1, 0). % rule 1 has head 1 and empty body

% domA(A) :- domA(A).
rule(2).

body(1).                 % body - identifier
body(1, 3).              % body has link to literal 1
function(3).             % symbol(function) - identifier
function_name(3, 1).     % name of function (shares the same name as function 1)
function_args(3, 2).     % link to arguments
args(2).                 % new arguments
args_pos_symb(2, 1, 4).  % arguments have symbol 4 at pos 1
variable(4).             % variable 4
variable_name(4, 2).     % link to name
name(2).                 % name identifier
name_repr(2, "A").       % repr of name

rule_head_body(2, 1, 1). % rule 2 has head 1 and body 1

"""

In [120]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0


In [121]:
ctl.add('base', [], unground_reified)

In [122]:
ctl.ground([('base', [])])

In [123]:
with ctl.solve(yield_=True) as solve_handle:
    models = []
    for model in solve_handle:
        symbols = model.symbols(atoms=True)
        models.append(symbols)

In [124]:
class Abstractifier:
    pos = clingo.ast.Position('<string>', 1, 1)
    loc = clingo.ast.Location(pos, pos)

    def __init__(self, model: Sequence[clingo.Symbol]):
        self.model: Sequence[clingo.Symbol] = model
        self._program_ast: Optional[Sequence[clingo.ast.AST]] = None

    @staticmethod
    def __handle(program_dict: dict, symbol: clingo.Symbol, workqueue: Optional[List[clingo.Symbol]] = None):
        fun_strs = dir(Abstractifier)
        name = symbol.name
        arity = len(symbol.arguments)
        fun_str = f'_Abstractifier__handle_{arity}_{name}'
        if fun_str in fun_strs:
            fun = getattr(Abstractifier, fun_str)
            return fun(program_dict, symbol, workqueue)
        return False

    @staticmethod
    def __handle_1_rule(program_dict: dict, symbol: clingo.Symbol, workqueue: Optional[List[clingo.Symbol]] = None):
        rule = symbol.arguments[0].number
        program_dict['rules'][rule] = {}
        return True

    @staticmethod
    def __handle_1_head(program_dict: dict, symbol: clingo.Symbol, workqueue: Optional[List[clingo.Symbol]] = None):
        head = symbol.arguments[0].number
        program_dict['heads'][head] = {}
        return True

    @staticmethod
    def __handle_1_body(program_dict: dict, symbol: clingo.Symbol, workqueue: Optional[List[clingo.Symbol]] = None):
        body = symbol.arguments[0].number
        program_dict['bodies'][body] = {}
        return True

    @staticmethod
    def __handle_1_literal(program_dict: dict, symbol: clingo.Symbol, workqueue: Optional[List[clingo.Symbol]] = None):
        literal = symbol.arguments[0].number
        program_dict['literals'][literal] = {}
        return True

    @staticmethod
    def __handle_1_atom(program_dict: dict, symbol: clingo.Symbol, workqueue: Optional[List[clingo.Symbol]] = None):
        atom = symbol.arguments[0].number
        program_dict['atoms'][atom] = {}
        return True

    @staticmethod
    def __handle_1_term(program_dict: dict, symbol: clingo.Symbol, workqueue: Optional[List[clingo.Symbol]] = None):
        term = symbol.arguments[0].number
        program_dict['terms'][term] = {}
        return True

    @staticmethod
    def __handle_1_function(program_dict: dict, symbol: clingo.Symbol, workqueue: Optional[List[clingo.Symbol]] = None):
        function = symbol.arguments[0].number
        program_dict['functions'][function] = {}
        return True

    @staticmethod
    def __handle_1_variable(program_dict: dict, symbol: clingo.Symbol, workqueue: Optional[List[clingo.Symbol]] = None):
        variable = symbol.arguments[0].number
        program_dict['variables'][variable] = {}
        return True

    @staticmethod
    def __handle_1_name(program_dict: dict, symbol: clingo.Symbol, workqueue: Optional[List[clingo.Symbol]] = None):
        name = symbol.arguments[0].number
        program_dict['names'][name] = {}
        return True

    @staticmethod
    def __handle_1_args(program_dict: dict, symbol: clingo.Symbol, workqueue: Optional[List[clingo.Symbol]] = None):
        args = symbol.arguments[0].number
        program_dict['args'][args] = {}
        return True

    @staticmethod
    def __handle_3_rule_head_body(program_dict: dict, symbol: clingo.Symbol,
                                  workqueue: Optional[List[clingo.Symbol]] = None):
        rule = symbol.arguments[0].number
        head = symbol.arguments[1].number
        body = symbol.arguments[2].number
        if rule not in program_dict['rules'] or head not in program_dict['heads'] or body not in program_dict['bodies']:
            workqueue.append(symbol)
            return False
        program_dict['rules'][rule]['head'] = head
        program_dict['rules'][rule]['body'] = body
        return True

    @staticmethod
    def __handle_2_literal_sign(program_dict: dict, symbol: clingo.Symbol,
                                workqueue: Optional[List[clingo.Symbol]] = None):
        literal = symbol.arguments[0].number
        sign = symbol.arguments[1].number
        if literal not in program_dict['literals']:
            workqueue.append(symbol)
            return False
        program_dict['literals'][literal]['sign'] = sign
        return True

    @staticmethod
    def __handle_2_literal_atom(program_dict: dict, symbol: clingo.Symbol,
                                workqueue: Optional[List[clingo.Symbol]] = None):
        literal = symbol.arguments[0].number
        atom = symbol.arguments[1].number
        if literal not in program_dict['literals'] or atom not in program_dict['atoms']:
            workqueue.append(symbol)
            return False
        program_dict['literals'][literal]['atom'] = atom
        return True

    @staticmethod
    def __handle_2_atom(program_dict: dict, symbol: clingo.Symbol, workqueue: Optional[List[clingo.Symbol]] = None):
        atom = symbol.arguments[0].number
        symb = symbol.arguments[1].number
        if atom not in program_dict['atoms'] or (
                symb not in program_dict['terms'] and
                symb not in program_dict['functions'] and
                symb not in program_dict['variables']
        ):
            workqueue.append(symbol)
            return False
        program_dict['atoms'][atom]['symb'] = symb
        return True

    @staticmethod
    def __handle_2_function_name(program_dict: dict, symbol: clingo.Symbol,
                                 workqueue: Optional[List[clingo.Symbol]] = None):
        function = symbol.arguments[0].number
        name = symbol.arguments[1].number
        if function not in program_dict['functions'] or name not in program_dict['names']:
            workqueue.append(symbol)
            return False
        program_dict['functions'][function]['name'] = name
        return True

    @staticmethod
    def __handle_2_name_repr(program_dict: dict, symbol: clingo.Symbol,
                             workqueue: Optional[List[clingo.Symbol]] = None):
        name = symbol.arguments[0].number
        repr = symbol.arguments[1].string
        if name not in program_dict['names']:
            workqueue.append(symbol)
            return False
        program_dict['names'][name]['repr'] = repr
        return True

    @staticmethod
    def __handle_2_head(program_dict: dict, symbol: clingo.Symbol, workqueue: Optional[List[clingo.Symbol]] = None):
        head = symbol.arguments[0].number
        child = symbol.arguments[1].number
        if head not in program_dict['heads'] or (
                child not in program_dict['literals']
        ):
            workqueue.append(symbol)
            return False
        program_dict['heads'][head]['child'] = child
        return True

    @staticmethod
    def __handle_2_term_repr(program_dict: dict, symbol: clingo.Symbol,
                             workqueue: Optional[List[clingo.Symbol]] = None):
        term = symbol.arguments[0].number
        repr = symbol.arguments[1]
        if term not in program_dict['terms']:
            workqueue.append(symbol)
            return False
        program_dict['terms'][term]['repr'] = repr
        return True

    @staticmethod
    def __handle_2_function_args(program_dict: dict, symbol: clingo.Symbol,
                                 workqueue: Optional[List[clingo.Symbol]] = None):
        function = symbol.arguments[0].number
        args = symbol.arguments[1]


    @staticmethod
    def __get_rule(program_dict: dict, rule: int):
        if 'AST' in program_dict['rules'][rule]:
            return program_dict['rules'][rule]['AST']
        head = program_dict['rules'][rule]['head']
        body = program_dict['rules'][rule]['body']
        head_node = Abstractifier.__get_head(program_dict, head)
        body_node = Abstractifier.__get_body(program_dict, body)
        program_dict['rules'][rule]['AST'] = clingo.ast.Rule(Abstractifier.loc, head=head_node, body=body_node)
        return program_dict['rules'][rule]['AST']

    @staticmethod
    def __get_head(program_dict: dict, head: int):
        if 'AST' in program_dict['heads'][head]:
            return program_dict['heads'][head]['AST']
        child = program_dict['heads'][head]['child']
        if child in program_dict['literals']:
            child_node = Abstractifier.__get_literal(program_dict, child)

        program_dict['heads'][head]['AST'] = child_node
        return program_dict['heads'][head]['AST']

    @staticmethod
    def __get_body(program_dict: dict, body: int):
        if 'AST' in program_dict['bodies'][body]:
            return program_dict['bodies'][body]['AST']
        return ()

    @staticmethod
    def __get_literal(program_dict: dict, literal: int):
        if 'AST' in program_dict['literals'][literal]:
            return program_dict['literals'][literal]['AST']

        sign = program_dict['literals'][literal]['sign']
        atom = program_dict['literals'][literal]['atom']
        atom_node = Abstractifier.__get_atom(program_dict, atom)
        literal_node = clingo.ast.Literal(Abstractifier.loc, sign, atom_node)
        program_dict['literals'][literal]['AST'] = literal_node
        return program_dict['literals'][literal]['AST']

    @staticmethod
    def __get_atom(program_dict: dict, atom: int):
        if 'AST' in program_dict['atoms'][atom]:
            return program_dict['atoms'][atom]['AST']

        symb = program_dict['atoms'][atom]['symb']
        if symb in program_dict['terms']:
            symb_node = Abstractifier.__get_term(program_dict, symb)
        elif symb in program_dict['functions']:
            symb_node = Abstractifier.__get_function(program_dict, symb)

        atom_node = clingo.ast.SymbolicAtom(symb_node)
        program_dict['atoms'][atom]['AST'] = atom_node
        return program_dict['atoms'][atom]['AST']

    @staticmethod
    def __get_term(program_dict: dict, term: int):
        if 'AST' in program_dict['terms'][term]:
            return program_dict['terms'][term]['AST']

        repr = program_dict['terms'][term]['repr']
        term_node = clingo.ast.SymbolicTerm(Abstractifier.loc, repr)
        program_dict['terms'][term]['AST'] = term_node
        return program_dict['terms'][term]['AST']

    @staticmethod
    def __get_function(program_dict: dict, fun: int):
        if 'AST' in program_dict['functions'][fun]:
            return program_dict['functions'][fun]['AST']

        name = program_dict['functions'][fun]['name']
        args = program_dict['functions'][fun]['args']
        fun_node = clingo.ast.Function(Abstractifier.loc, name, args, False)
        program_dict['functions'][fun]['AST'] = fun_node
        return program_dict['functions'][fun]['AST']



    def build(self):
        program_dict = dict(rules={},
                            heads={0: dict(AST=clingo.ast.BooleanConstant(0))},
                            bodies={0: dict(AST=())},
                            literals={},
                            terms={},
                            atoms={},
                            functions={},
                            variables={},
                            names={},
                            args={},
                            )
        workqueue = []
        for symbol in self.model:
            Abstractifier.__handle(program_dict, symbol, workqueue)

        skips = 0
        while workqueue and len(workqueue) > skips:
            symbol = workqueue.pop(0)
            worked = Abstractifier.__handle(program_dict, symbol, workqueue)
            if worked:
                skips = 0
            else:
                skips += 1
        if workqueue:
            print(f"Could not parse model after {skips} skips.")
            print(workqueue)

        print(program_dict)
        nodes = []
        for rule in program_dict['rules']:
            node = Abstractifier.__get_rule(program_dict, rule)
            # head = program_dict['rules'][rule]['head']
            # body = program_dict['rules'][rule]['body']
            # head_node = clingo.ast.BooleanConstant(0)
            # body_node = []
            # if head > 0:
            #     child = program_dict['heads'][head]['child']
            #     if child in program_dict['literals']:
            #         literal = child
            #         sign = program_dict['literals'][literal]['sign']
            #         atom = program_dict['literals'][literal]['atom']
            #         symb = program_dict['atoms'][atom]['symb']
            #         if symb in program_dict['functions']:
            #             name = program_dict['functions'][symb]['name']
            #             repr = program_dict['names'][name]['repr']
            #             symb_node = clingo.ast.Function(Abstractifier.loc, repr, [], False)
            #         elif symb in program_dict['terms']:
            #             pass
            #         elif symb in program_dict['variables']:
            #             pass
            #         atom_node = clingo.ast.SymbolicAtom(symb_node)
            #         head_node = clingo.ast.Literal(Abstractifier.loc, sign, atom_node)
            #
            # if body > 0:
            #     pass
            #
            # node = clingo.ast.Rule(Abstractifier.loc, head=head_node, body=body_node)
            nodes.append(node)

        self._program_ast = nodes

    def program(self):
        if self._program_ast is None:
            self.build()
        return self._program_ast

In [125]:
abstractifier = Abstractifier(models[0])
abstractifier.build()
prg = abstractifier.program()

{'rules': {1: {'head': 1, 'body': 0}, 2: {'head': 1, 'body': 1}}, 'heads': {0: {'AST': ast.BooleanConstant(0)}, 1: {'child': 1}}, 'bodies': {0: {'AST': ()}, 1: {}}, 'literals': {1: {'sign': 0, 'atom': 1}}, 'terms': {2: {'repr': Number(0)}}, 'atoms': {1: {'symb': 1}}, 'functions': {1: {'name': 1}, 3: {'name': 1}}, 'variables': {4: {}}, 'names': {1: {'repr': 'domA'}, 2: {'repr': 'A'}}, 'args': {1: {}, 2: {}}}


KeyError: 'args'

In [ ]:
print('\n'.join(map(str, prg)))

In [ ]:
pos = clingo.ast.Position('<string>', 1, 1)
loc = clingo.ast.Location(pos, pos)